VacationPy

In [23]:
# dependencies
import hvplot.pandas
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import time


In [3]:
# import API key
from api_keys import geoapify_api_key

In [5]:
# load the WeatherPy_Database.csv file
file_path = "/Users/grahammarsh/Documents/GitHub/python-api-challenge/Final Analysis/FinalOutPutData/WeatherPy_Database.csv"
weather_df = pd.read_csv(file_path)

# display the header of the weather_df
weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ribeira Grande,38.5167,-28.7000,17.65,77,40,10.29,PT,1707747506
1,1,Saint-Philippe,-21.3585,55.7679,28.77,86,60,5.10,RE,1707747506
2,2,Codrington,-38.2667,141.9667,14.44,92,0,2.67,AU,1707747507
3,3,Waitangi,-43.9535,-176.5597,12.68,87,52,0.89,NZ,1707747507
4,4,Port-Aux-Francais,-49.3500,70.2167,6.91,85,100,14.40,TF,1707747507


Step 1: Create a map that displays a point for every city in the weather_df DataFrame. The size of the point represents the humidity level in each city.

In [24]:
# a world map depicting cities and humidity from the weather_df
# create a map
weather_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    frame_height=500,
    frame_width=700,
    color="City"
)



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

Step 2: Narrow down the weather_df DataFrame to find your ideal weather condition

In [12]:
# filter the weather_df for cities with humidity levels less than 70%
low_humidity_df = weather_df.loc[weather_df["Humidity"] < 70]
low_humidity_df.head()

# drop ay rows with NaN values
low_humidity_df = low_humidity_df.dropna()
low_humidity_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,Hasaki,35.7333,140.8333,2.94,45,0,1.70,JP,1707747508
11,11,Barra De Santo Antonio,-9.4047,-35.5072,27.60,55,63,5.87,BR,1707747508
15,15,Puerto Natales,-51.7236,-72.4875,12.25,54,75,8.75,CL,1707747391
19,19,Minas De Marcona,-15.2119,-75.1103,21.78,66,51,4.59,PE,1707747509
24,24,Iqaluit,63.7506,-68.5145,-29.06,68,20,1.03,CA,1707747510


Step 3: Create a new DataFrame called hotel_df

In [13]:
# use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates and humidity
hotel_df = low_humidity_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# add a new column to the hotel_df called "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
9,Hasaki,JP,35.7333,140.8333,45,
11,Barra De Santo Antonio,BR,-9.4047,-35.5072,55,
15,Puerto Natales,CL,-51.7236,-72.4875,54,
19,Minas De Marcona,PE,-15.2119,-75.1103,66,
24,Iqaluit,CA,63.7506,-68.5145,68,


Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

In [9]:
# define the search categories
categories = "accommodation.hotel"

# define the search radius
radius = 10000

# define the search limit
limit = 1

# define the parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_api_key
}

# print a message to follow up the hotel search
print("Starting hotel search")

# iterate through the hotel_df and retrieve the hotel name for each city
for index, row in hotel_df.iterrows():
    # get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add the latitude and longitude to the parameters dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # set the base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # run the API request using the parameters dictionary and convert the response to JSON
    name_address = requests.get(base_url, params=params).json()
    
    # get the first hotel name from the response and store it in the hotel_df
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (IndexError, KeyError):
        hotel_df.loc[index, "Hotel Name"] = "Hotel not found"

    # log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# display the hotel_df
hotel_df


Starting hotel search
Hasaki - nearest hotel: 大新旅館
Barra De Santo Antonio - nearest hotel: Resort Fazenda Fiore
Puerto Natales - nearest hotel: Dorotea Patagonia Hostel
Minas De Marcona - nearest hotel: Hotel not found
Iqaluit - nearest hotel: Frobisher Inn
Isafjordur - nearest hotel: Hótel Horn
Basco - nearest hotel: Tawsen Hotel
Punta Arenas - nearest hotel: Hotel Lacolet
Baiyin - nearest hotel: 红鹭宾馆
Kavaratti - nearest hotel: Hotel not found
Arraial Do Cabo - nearest hotel: Hotel not found
Seguela - nearest hotel: Hotel not found
Margate - nearest hotel: The Reading Rooms
Bredasdorp - nearest hotel: Victoria Hotel
Whitehorse - nearest hotel: Town & Mountain Hotel
Ancud - nearest hotel: Cabañas
Ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
Ayame - nearest hotel: Hotel not found
Tamanrasset - nearest hotel: فندق الأمان
Sao Raimundo Das Mangabeiras - nearest hotel: Hotel not found
As Sulayyil - nearest hotel: Hotel not found
Awjilah - nearest hotel: Hotel not found
Port 

,City,Country,Lat,Lng,Humidity,Hotel Name
9,Hasaki,JP,35.7333,140.8333,45,大新旅館
11,Barra De Santo Antonio,BR,-9.4047,-35.5072,55,Resort Fazenda Fiore
15,Puerto Natales,CL,-51.7236,-72.4875,54,Dorotea Patagonia Hostel
19,Minas De Marcona,PE,-15.2119,-75.1103,66,Hotel not found
24,Iqaluit,CA,63.7506,-68.5145,68,Frobisher Inn
...,...,...,...,...,...,...
586,Altos,BR,-5.0381,-42.4600,55,Hotel not found
588,Recreo,AR,-29.2818,-65.0610,38,Hotel Sarmiento
590,Richards Bay,ZA,-28.7830,32.0377,55,SUN1 Richards Bay
591,Paita,PE,-5.0892,-81.1144,67,Victor's


Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
# configure the map plot
hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_height=500,
    frame_width=700,
    color="Hotel Name",
    hover_cols=["City", "Country", "Hotel Name"]
)



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)